In [237]:
# Lets first load in the relevant yearly files
import pickle

def keep_duplicate_player_most_minutes(input_df):
    idx_to_keep = []
    for curr_player in input_df['Player']:
        check_dup = input_df[['Player','MP']][input_df['Player'] == curr_player]
        idx_to_keep.append(check_dup['MP'].idxmax())
    idx_to_keep = np.unique(idx_to_keep)
    output_df = input_df.loc[list(idx_to_keep)]
    return output_df

with open('/Users/rohanramesh/Documents/SportsData/NBA/YearlyBasicData.pickle', 'rb') as handle:
    Yearly_df = pickle.load(handle)

    
with open('/Users/rohanramesh/Documents/SportsData/NBA/YearlyAdvData.pickle', 'rb') as handle:
    Yearly_df_adv = pickle.load(handle)


print(Yearly_df['2018'])

      2P    2P%   2PA   3P    3P%  3PA  AST   Age  BLK  DRB    ...     TOV  \
0    0.4  0.443   0.9  1.1  0.380  2.9  0.4  24.0  0.1  1.2    ...     0.3   
1    0.4  0.384   1.0  1.5  0.349  4.2  0.8  27.0  0.4  3.1    ...     0.9   
2    5.9  0.631   9.3  0.0  0.000  0.0  1.2  24.0  1.0  4.0    ...     1.7   
3    2.5  0.523   4.8  0.0  0.000  0.1  1.5  20.0  0.6  3.8    ...     1.0   
4    0.7  0.413   1.7  0.5  0.386  1.3  0.6  32.0  0.2  1.2    ...     0.4   
5    0.2  0.333   0.7  0.0  0.000  0.0  0.1  29.0  0.0  0.6    ...     0.0   
6    8.8  0.526  16.7  0.4  0.293  1.2  2.0  32.0  1.2  5.2    ...     1.5   
7    3.2  0.599   5.3  0.1  0.333  0.2  0.7  19.0  1.2  3.4    ...     1.1   
8    0.3  0.545   0.6  0.0  0.000  0.6  0.7  25.0  0.1  0.4    ...     0.5   
9    1.8  0.506   3.6  0.2  0.333  0.5  0.4  36.0  0.1  1.2    ...     0.9   
10   1.5  0.432   3.5  1.8  0.369  4.9  1.2  27.0  0.6  6.2    ...     1.1   
11   1.4  0.535   2.6  0.9  0.330  2.7  0.7  24.0  0.2  1.8    .

In [238]:
# remove doubles for each year
for curr_year in range(2010,2019):
    print(curr_year)
    Yearly_df[str(curr_year)] = keep_duplicate_player_most_minutes(Yearly_df[str(curr_year)])
    Yearly_df_adv[str(curr_year)] = keep_duplicate_player_most_minutes(Yearly_df_adv[str(curr_year)])

2010
2011
2012
2013
2014
2015
2016
2017
2018


In [167]:
# Lets look at pairs of years and find those positional players that had the largest jump in some statistical category
import numpy as np
import pandas as pd
curr_basic_header = list(Yearly_df['2018'])
curr_adv_header = list(Yearly_df_adv['2018'])
stat_to_consider = 'PER'
stats_include = ['Player', stat_to_consider, 'MP', 'Pos']
position_to_test = 'PF'
for curr_year in range(2018,2017,-1):
    # lets first declare a position
    idx = (Yearly_df_adv[str(curr_year)]['Pos'] == position_to_test) & (Yearly_df_adv[str(curr_year)]['MP'] > 400)
    curr_stat = Yearly_df_adv[str(curr_year)][stats_include][idx]
    curr_stat = keep_duplicate_player_most_minutes(curr_stat) # remove duplicates
    # for previous year don't care about what position was
    idx = (Yearly_df_adv[str(curr_year-1)]['MP'] > 400)
    previous_yr_stat = Yearly_df_adv[str(curr_year-1)][stats_include][idx]
    previous_yr_stat = keep_duplicate_player_most_minutes(previous_yr_stat)
    # now for each player iterate through and look the year before for that player
    print(np.shape(curr_stat))
    stat_difference = []
    for curr_player in curr_stat['Player']:
        prev_stat = previous_yr_stat[stats_include][previous_yr_stat['Player'] == curr_player]
        if prev_stat.empty:
            print(curr_player)
        else:
#             print(prev_stat)
            stat_player_curr_year = curr_stat[stat_to_consider][curr_stat['Player'] == curr_player].tolist()
            stat_player_last_year = prev_stat[stat_to_consider].tolist()
            print(stat_player_curr_year)
            print(stat_player_last_year)
            print(stat_player_curr_year[0]-stat_player_last_year[0])
#             stat_difference.append(stat_player_curr_year-prev_stat[stat_to_consider])
#     print(stat_difference)
    

(69, 4)
[8.2]
[11.8]
-3.6000000000000014
[12.0]
[11.3]
0.6999999999999993
[12.6]
[13.5]
-0.9000000000000004
[27.3]
[26.1]
1.1999999999999993
[12.7]
[17.9]
-5.199999999999999
[15.8]
[16.3]
-0.5
[17.8]
[17.8]
0.0
[7.1]
[5.3]
1.7999999999999998
[14.1]
[12.9]
1.1999999999999993
[12.6]
[11.0]
1.5999999999999996
[14.9]
[15.7]
-0.7999999999999989
Tyler Cavanaugh
[11.1]
[12.3]
-1.200000000000001
John Collins
[28.9]
[27.5]
1.3999999999999986
[11.5]
[13.1]
-1.5999999999999996
Cheick Diallo
[8.8]
[10.6]
-1.799999999999999
[26.0]
[27.6]
-1.6000000000000014
[17.4]
[20.3]
-2.900000000000002
[7.6]
[7.7]
-0.10000000000000053
[18.0]
[17.9]
0.10000000000000142
[15.4]
[15.0]
0.40000000000000036
[16.5]
[14.5]
2.0
[16.2]
[10.1]
6.1
[16.1]
[16.5]
-0.3999999999999986
[14.0]
[13.5]
0.5
[14.8]
[10.5]
4.300000000000001
[19.6]
[22.6]
-3.0
[17.1]
[16.9]
0.20000000000000284
Josh Huestis
[14.7]
[16.4]
-1.6999999999999993
[14.9]
[14.6]
0.3000000000000007
Jonathan Isaac
[28.6]
[27.0]
1.6000000000000014
[13.2]
[9.3]
3

In [213]:
print(list(Yearly_df_adv['2018']))

['3PAr', 'AST%', 'Age', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'G', 'MP', 'OBPM', 'ORB%', 'OWS', 'PER', 'Player', 'Pos', 'STL%', 'TOV%', 'TRB%', 'TS%', 'Tm', 'USG%', 'VORP', 'WS', 'WS/48', '\xa0']


In [85]:
curr_player = 'Quincy Acy'
# print(previous_yr_stat['Player'] == curr_player)
tmpR = previous_yr_stat[['Player', 'PER', 'MP']][previous_yr_stat['Player'] == curr_player]
print(np.shape(tmpR)[0])

2


In [214]:
print(Yearly_df['2018']['Player'])

0               Alex Abrines
1                 Quincy Acy
2               Steven Adams
3                Bam Adebayo
4              Arron Afflalo
5               Cole Aldrich
6          LaMarcus Aldridge
7              Jarrett Allen
8               Kadeem Allen
9                 Tony Allen
10           Al-Farouq Aminu
11           Justin Anderson
12             Kyle Anderson
13             Ryan Anderson
14              Ike Anigbogu
15     Giannis Antetokounmpo
16           Carmelo Anthony
17                OG Anunoby
18          Ryan Arcidiacono
19              Trevor Ariza
20            Darrell Arthur
21               Jamel Artis
24                 Omer Asik
25             D.J. Augustin
27              Luke Babbitt
29              Dwayne Bacon
30                 Ron Baker
31              Wade Baldwin
32                Lonzo Ball
33                J.J. Barea
               ...          
628              Okaro White
629         Isaiah Whitehead
630         Hassan Whiteside
631           

In [241]:
import numpy as np
import pandas as pd
# build a function that will add a category that looks at the change in a stat from the previous year
def add_column_for_change_in_stat(df,stat_to_consider):
    # lets get the years to consider
    all_years = [int(i) for i in list(df)[::-1]]  
    stats_include = ['Player', stat_to_consider]
    for curr_year in all_years[0:-1]:
        # current year stat
        curr_stat = df[str(curr_year)][stats_include]
        # for previous year don't care about what position was
        previous_yr_stat = df[str(curr_year-1)][stats_include]
        # now for each player iterate through and look the year before for that player
        stat_difference = []
        for curr_player in curr_stat['Player']:
            prev_stat = previous_yr_stat[stats_include][previous_yr_stat['Player'] == curr_player]
            if prev_stat.empty:
    #             print(curr_player)
                stat_difference.append(np.nan)
            else:
                stat_player_curr_year = curr_stat[stat_to_consider][curr_stat['Player'] == curr_player].tolist()
                stat_player_last_year = prev_stat[stat_to_consider].tolist()
                stat_difference.append(stat_player_curr_year[0]-stat_player_last_year[0])
        # now define this as a dict so that can append into dataframe
        df[str(curr_year)]['d'+stat_to_consider] = stat_difference
    return df


In [240]:
# add the change in stats to the adv stats
stats_to_look_at_increment = ['PER','VORP','USG%','WS']
for curr_stat in stats_to_look_at_increment:
    Yearly_df_adv = add_column_for_change_in_stat(Yearly_df_adv,curr_stat)


[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]
2018
2017
2016
2015
2014
2013
2012
2011
[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]
2018
2017
2016
2015
2014
2013
2012
2011
[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]
2018
2017
2016
2015
2014
2013
2012
2011
[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011]
2018
2017
2016
2015
2014
2013
2012
2011


In [228]:
print(list(Yearly_df_adv['2018']))
print(range(2018,2010,-1))

['3PAr', 'AST%', 'Age', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'G', 'MP', 'OBPM', 'ORB%', 'OWS', 'PER', 'Player', 'Pos', 'STL%', 'TOV%', 'TRB%', 'TS%', 'Tm', 'USG%', 'VORP', 'WS', 'WS/48', '\xa0', 'dPER', 'dVORP', 'dUSG%', 'dWS']
range(2018, 2010, -1)
